In [1]:
import spacy
import pandas as pd
import numpy as np


## モデルのロードとデータの読み込み
nlp = spacy.load("ja_ginza")
df = pd.read_excel("sisyou_db_h29_01.xlsx", skiprows=[1])
df.head()

,ID,年号,年,月,発生\n時間,災害状況,業種（大分類）,Unnamed: 7,業種（中分類）,Unnamed: 9,...,事業場\n規模,起因物（大分類）,Unnamed: 14,起因物（中分類）,Unnamed: 16,起因物（小分類）,Unnamed: 18,事故の型,Unnamed: 20,年齢
0,1,平成,29,1,22～23,工場内で床に置いていたコードに、荷物を抱えていた状態のときに足が引っ掛かり、よろめいて数歩前...,1,製造業,108,化学工業,...,―,3,その他の装置等,37,用具,379,その他の用具,3,激突,54
1,2,平成,29,1,19～20,倉庫の出入口の階段を荷物（冷凍商品15kgぐらい）を持って下りる際に、階段が凍っていて滑って...,8,商業,804,その他の商業,...,100～299,7,環境等,71,環境等,719,その他の環境等,2,転倒,39
2,3,平成,29,1,18～19,会社構内にて車輌の洗車中、足を滑らせ転倒した際に左手をつき、翌朝に左肩の痛みが大きくなり、左...,4,運輸交通業,403,道路貨物運送業,...,100～299,7,環境等,71,環境等,719,その他の環境等,2,転倒,54
3,4,平成,29,1,16～17,厩舎2階でバッカン受け入れ作業中、バッカンを落とす穴から落下した。,7,畜産・水産業,701,畜産業,...,10～29,4,仮設物、建築物、構築物等,41,仮設物、建築物、構築物等,414,開口部,1,墜落、転落,50
4,5,平成,29,1,16～17,勤務先の食堂施設内で、ダンボールを束ねてビニールの荷造り紐で縛り結んだ時、手が滑り勢いよく壁...,8,商業,802,小売業,...,50～99,4,仮設物、建築物、構築物等,41,仮設物、建築物、構築物等,419,その他の仮設物、建築物、構築物等,6,激突され,65


In [2]:
## 文書ベクトルの表示
text = df["災害状況"][0]
print(text)

doc = nlp(text)
print(doc.vector)

工場内で床に置いていたコードに、荷物を抱えていた状態のときに足が引っ掛かり、よろめいて数歩前に進んだのち、前方にあった作業台に衝突して受傷した。
[ 0.00328202 -0.12495309  0.03104955 -0.10209632 -0.07832336  0.02880605
 -0.08283816 -0.04763658 -0.168323   -0.0778297   0.05533801 -0.0986171
  0.0005385   0.02962441 -0.1167674  -0.18914986 -0.09882364 -0.09396566
 -0.08596961  0.05919072 -0.10210434  0.12087278  0.02043262  0.078091
 -0.10516028 -0.12018455 -0.17736383  0.04216171  0.00655846  0.06014617
 -0.00634867 -0.07965478  0.14818329  0.1414089   0.00074081  0.00370849
 -0.07824669  0.08709563 -0.06225902  0.01427439  0.08103152  0.02889493
 -0.08473103 -0.05481999 -0.12609008 -0.07398206  0.07231876  0.00028355
 -0.12315308  0.07644407 -0.00290333  0.02831611  0.02464396  0.05821903
  0.03150168 -0.03997846  0.0307556   0.013209   -0.06771619 -0.00887364
 -0.02989043  0.04266423 -0.01720139 -0.08101492 -0.05531686 -0.04679136
  0.13107067  0.20535089  0.10728855 -0.03173365  0.01891442  0.06155709
 -0.02212167  0.01199526 -0.06953648  0.01438895  0.04

In [3]:
## 文書間類似度の算出
texts = [
    "工場内で、鉄骨階段（仮組）高さ2m50cmに手摺を仮組立作業中、足を滑らせて転倒し、顔面を強打して骨折し、右脇腹肋骨も骨折した。",
    "倉庫の出入口の階段を荷物（冷凍商品15kgぐらい）を持って下りる際に、階段が凍っていて滑って転倒し、階段を転げ落ち（4段位）、持っていた荷物を足に落としてしまい、右足の腓骨を骨折した。",
    "道路の3車線の真ん中を走行中、左車線に侵入してしまい、走行中の大型ワンボックスカーと衝突し、首と左肩を痛め、回転性のめまいで入院し、痺れもある。"
]
docs = list(nlp.pipe(texts))

for i in range(len(docs)-1):
    for j in range(i+1, len(docs)):
        print(i, j, docs[i].similarity(docs[j]))

0 1 0.9421024573246568
0 2 0.8961613533334881
1 2 0.897050319104168


In [4]:
## 文書間類似度行列の作成
from sklearn.preprocessing import normalize

vecs = [doc.vector for doc in nlp.pipe(df["災害状況"])]
X = np.array(vecs)
X = normalize(X, axis=1, norm='l2')

cos_sims = X @ X.T
print(cos_sims)

[[1.0000002  0.96133286 0.93254447 ... 0.89231026 0.9658282  0.9476143 ]
 [0.96133286 1.0000001  0.93941194 ... 0.8845368  0.95780617 0.93922675]
 [0.93254447 0.93941194 1.         ... 0.92859584 0.94712013 0.9256242 ]
 ...
 [0.89231026 0.8845368  0.92859584 ... 1.0000002  0.8997892  0.8929449 ]
 [0.9658282  0.95780617 0.94712013 ... 0.8997892  0.9999999  0.9433455 ]
 [0.9476143  0.93922675 0.9256242  ... 0.8929449  0.9433455  0.9999998 ]]


In [5]:
## 類似度行列を用いて類似文書検索
target = 9
print(target, df["災害状況"][target])

top_index = np.argsort(cos_sims[target])[::-1]
print(df["災害状況"][top_index][1:10]) #top_index[0]は入力文書

9 棚卸作業をしていたところ、ラムダテナー（資材用ラック）2段目にかけた梯子に足を掛けた際に、梯子がずれ落ちてバランスを崩し、右片足で着地した。
外傷は特に無かったが、右足を地面に着けて立ち上がる事が出来なくなり、右足アキレス腱断裂と診断された。
2288    昼食のお膳を片手に1人分ずつ（計2人分）持ち運んだ際、部屋の入口付近で床に躓きバランスが崩れ...
1756    売場業務中、売場外から中へ入る時、レジ横小扉を開け、中に入ろうとした時、扉の下側に左足が引っ...
305     荷卸し終了後にトラックの荷台より降りようとした際、足を滑らせて地面へ転倒し、左腰後ろの部分を...
2590    朝、職員昇降口から入り、廊下を小走りで更衣室に向かっているとき、左足を捻った。\n咄嗟に廊下...
800     \n左足を軽トラックの荷台に載せ、右足をフロアにつけた状態で作業をしていたが、工具箱がフロア...
2447    下駄箱前で靴を履きかえた後、立ち上がる際にバランスを崩して左側に転倒し、左大腿部のあたりを強...
1477    30kgくらいの荷物を3階へ運んでいる際に足場が確認できず、不安定な状態で足が着地したために...
2213    1番ホールド内で脚立、ローリングタワーが使用不可能な場所でパイプ上に乗り、養生はずしの作業を...
153     事業所内2階から1階へ階段を下りたところ、階段のすぐ前に一時的に置いてあった使用済みタオル袋...
Name: 災害状況, dtype: object
